In [1]:
import pandas as pd

import requests
from bs4 import BeautifulSoup

BASE: str = 'https://www.metacritic.com'
API: str = 'browse/movies/score/metascore/all/filtered'
QUERY = lambda n: f'sort=desc&page={n}'

AGENT: str = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36'

In [2]:
def scrape_page(num: int):
    url: str = f'{BASE}/{API}?{QUERY(num)}'

    s = requests.Session()
    s.headers['User-Agent'] = AGENT

    page = s.get(url, allow_redirects=True)
    soup = BeautifulSoup(page.content, "html.parser")

    return pd.DataFrame.from_records([
        {
            'title': movie.find_all('h3')[0].text,
            'score': movie.find_all('div', class_='metascore_w')[0].text,
            'summary': movie.find_all('div', class_='summary')[0].text
        }
        for movie in soup.find_all("tr") if movie.get('class', [None])[0] != 'spacer'
    ])

In [3]:
pd.concat(
    [scrape_page(n) for n in range(10)],
    ignore_index=True
)

,title,score,summary
0,The Godfather,100,\n Francis Ford Coppola...
1,Citizen Kane,100,\n Following the death ...
2,Rear Window,100,\n A wheelchair-bound p...
3,Casablanca,100,"\n A Casablanca, Morocc..."
4,Boyhood,100,\n Filmed over 12 years...
...,...,...,...
995,A Room with a View,83,\n Based on E.M. Forste...
996,Hugo,83,\n Based on Brian Selzn...
997,Tenebrae,83,\n An American writer i...
998,The LEGO Movie,83,\n An ordinary LEGO min...
